# Generate the SysML Vocabulary Terms

Generate the SysML v2 vocabulary from /Users/jamsden/Developer/SysML/SysML-v2-Pilot-Implementation/org.omg.sysml/model/SysML.ecore. This will be the merged KerML and SysML into a single vocabulary with a single namespace.

Set genOASIS = True to generate for OASIS, leave false to generate for OMG



In [1]:
# Initial imports and local methods
from rdflib import Graph, URIRef, Literal, Namespace, RDF, XSD, RDFS, OWL, DCTERMS, BNode
from pyecore.ecore import EClass, EAttribute, EReference, EString, EObject, EEnum
from pyecore.resources import ResourceSet, URI
from bs4 import BeautifulSoup
import os

genOASIS = False  # set to True to generate OASIS SysML instead of OMG: Deprecated, only generate OMG

# use the class name as a prefix to the attribute name to ensure they are unique
# this should no longer be used, but the method to calculate the qualified name is retained.
def qualident(eClass, eAttribute):
    return eClass.name[0].lower() + eClass.name[1:] + '_' + eAttribute.name[0].capitalize() + eAttribute.name[1:]  

# Get the description of the model element as the first paragraph
# with markdown removed if strip is True (rdfs:comment is a string, not an XMLLiteral)
def comment(eModelElement, strip=True):
    desc = f'{eModelElement.name}.'  # rdfs:comment is required in the vocabulary, this is the default
    a = eModelElement.getEAnnotation('http://www.eclipse.org/emf/2002/GenModel')
    if a is None: return desc
    description = a.details['documentation']
    root = BeautifulSoup(str(description))
    description = root.find('p') 
    if strip:
        text =  description.get_text() # the first paragraph with the HTML stripped out
        if not (text is None or text==''): desc = text
    else:
        # strip off the <p> and </p>, they are not needed and cause the period to appear to be missing to shapechecker
        desc = str(description)[3:-4]  # but include the rest of the markup
    if not desc.endswith('.'): desc = desc + '.'
    return desc


# some useful RDF namespaces
vann = Namespace('http://purl.org/vocab/vann/')
oslc = Namespace('http://open-services.net/ns/core#')
oslc_am = Namespace('http://open-services.net/ns/am#')
oslc_sysml = Namespace('https://www.omg.org/spec/sysml/vocabulary#')  # OMG namespace
if genOASIS: oslc_sysml = Namespace('http://open-services.net/ns/sysml#')  # OASIS namespace

# the OASIS copywrite and license information, not used for OMG 
copyright = """
# Copyright 2024 OASIS Open
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""


In [ ]:
# Generate the SysML OSLC vocabulary file

vocab_file = 'SysML-vocab.ttl'
if genOASIS: vocab_file = 'sysml-vocab.ttl'

# create the SysML vocabulary graph, initially empty
g = Graph()
g.bind('oslc_sysml', oslc_sysml)
g.bind('oslc_am', oslc_am)

# add the vocabulary ontology
o = URIRef(oslc_sysml)
g.add((o, RDF.type, OWL.Ontology))
g.add((o, RDFS.label, Literal("OSLC SysML v2 Vocabulary")))
g.add((o, DCTERMS.description, Literal("All vocabulary URIs defined in the OSLC SysML v2 namespace.", datatype=RDF.XMLLiteral)))
g.add((o, DCTERMS.title, Literal("OSLC SysML v2 Vocabulary")))
g.add((o, vann.preferredNamespacePrefix, Literal("oslc_sysml")))
if genOASIS:
    g.add((o, DCTERMS.dateCopyrighted, Literal("2012-2024")))
    g.add((o, DCTERMS.hasVersion, Literal("PSD01")))
    g.add((o, DCTERMS.isPartOf, URIRef('https://docs.oasis-open-projects.org/oslc-op/sysml/v2.0/os/sysml-spec.html')))
    g.add((o, DCTERMS.issued, Literal("2024-08-01", datatype=XSD.date)))
    g.add((o, DCTERMS.license, URIRef('http://www.apache.org/licenses/LICENSE-2.0')))
    g.add((o, DCTERMS.publisher,  URIRef('https://open-services.net/about/')))
    g.add((o, DCTERMS.source, URIRef('https://docs.oasis-open-projects.org/oslc-op/sysml/v2.0/os/sysml-vocab.ttl')))
else:
    g.add((o, DCTERMS.publisher,  URIRef('https://www.omg.org')))
    g.add((o, DCTERMS.issued, Literal("2025-02-01", datatype=XSD.date)))
    g.add((o, DCTERMS.hasVersion, Literal("V2")))
    g.add((o, DCTERMS.isPartOf, URIRef('https://www.omg.org/spec/SysML')))
    g.add((o, DCTERMS.source, URIRef('https://github.com/Systems-Modeling/SysML-v2-API-Services/tree/master/public/oslc/SysML-vocab.ttl')))
    g.add((o, DCTERMS.license, URIRef('')))

# load the SysML.ecore model - this is the merged KerML and SysML metamodel using a single namespace
rset = ResourceSet()
resource = rset.get_resource(URI('/Users/jamsden/Developer/SysML-v2-Pilot-Implementation/org.omg.sysml/model/sysml_clean.ecore'))
mm_root = resource.contents[0]
rset.metamodel_registry[mm_root.nsURI] = mm_root
# At this point, the .ecore is loaded in the 'rset' as a metamodel


for c in mm_root.eClassifiers:
    # Generate the classes
    if isinstance(c, EClass):
        g.add((oslc_sysml.term(c.name), RDF.type, RDFS.Class))
        if c.name=='Element':
            g.add((oslc_sysml.term(c.name), RDFS.subClassOf, oslc_am.Resource))  # only Element
        if c.eSuperTypes is not None and len(c.eSuperTypes) >= 1:
            for super in c.eSuperTypes:
                g.add((oslc_sysml.term(c.name), RDFS.subClassOf, oslc_sysml.term(super.name)))
        g.add((oslc_sysml.term(c.name), RDFS.label, Literal(c.name)))
        g.add((oslc_sysml.term(c.name), RDFS.comment, Literal(comment(c))))
        g.add((oslc_sysml.term(c.name), RDFS.isDefinedBy, URIRef(oslc_sysml)))

        # generate the properties
        for a in c.eStructuralFeatures:
            name = a.name  # done't use the class name as a prefix to the attribute name to ensure they are unique
            s = oslc_sysml.term(name)
            g.add((s, RDF.type, RDF.Property))
            g.add((s, RDFS.label, Literal(name)))
            # does the property already have a comment?
            pcomment = g.value(s, RDFS.comment)
            if pcomment is not None:
                # This is a property with multiple domains
                # Concatenate the comment for each domain class into a single comment
                pcomment = pcomment + '\n' + c.name+': '+comment(a)
                g.remove((s, RDFS.comment, None))
                g.add((s, RDFS.comment, Literal(pcomment)))
            else:
                g.add((s, RDFS.comment, Literal(c.name+': '+comment(a))))
            g.add((s, RDFS.isDefinedBy, URIRef(oslc_sysml)))
    if isinstance(c, EEnum):
        g.add((oslc_sysml.term(c.name), RDF.type, RDFS.Class))
        g.add((oslc_sysml.term(c.name), RDFS.label, Literal(c.name)))
        g.add((oslc_sysml.term(c.name), RDFS.comment, Literal(comment(c))))
        g.add((oslc_sysml.term(c.name), RDFS.isDefinedBy, URIRef(oslc_sysml)))

        # generate the enumeration literals
        for a in c.eLiterals:
            name = a.name  # don't use the class name as a prefix to the attribute name to ensure they are unique
            s = oslc_sysml.term(name)
            g.add((s, RDF.type, oslc_sysml.term(c.name)))
            g.add((s, RDFS.label, Literal(name)))
            g.add((s, RDFS.comment, Literal(comment(a))))
            g.add((s, RDFS.isDefinedBy, URIRef(oslc_sysml)))
        
g.serialize(destination=f'../{vocab_file}')

if genOASIS:  # add the copyright for OASIS
    with open('../sysml-vocab.ttl','r') as f:
        with open('../temp.ttl','w') as f2: 
            f2.write(copyright)
            f2.write(f.read())
    os.remove('../sysml-vocab.ttl')
    os.rename('../temp.ttl','../sysml-vocab.ttl')

# Generate the SysML OSLC constraints
Generate the SysML v2 constraints from /Users/jamsden/Developer/SysML/SysML-v2-Pilot-Implementation/org.omg.sysml/model/SysML.ecore. This will be the merged KerML and SysML into a single vocabulary with a single namespace. 

This depends on the vocabulary that was generated above.

The ResourceShape properties also do not need class prefixes because they are added to the classes using blank nodes which makes them unique.



In [3]:
# Get a list of the properties that are defined in more than one class (ignoring inheritance)
# These need to be added as shape properties using blank nodes to ensure they are properly scoped by the class
# Iterate over all the EClasses in the model and collect the EAttribute and EReference property names
# Then find all the ones that are duplicated
import pandas as pd

allProperties = []
for c in mm_root.eClassifiers:
    if isinstance(c, EClass):
         for a in c.eStructuralFeatures:
              allProperties.append(a.name)
allProperties = pd.Series(allProperties)
multiDefProps = set(allProperties[allProperties.duplicated()]);

In [ ]:
# Generate the SysML OSLC constraints
from rdflib import Graph, URIRef, Literal, Namespace, RDF, XSD, RDFS, OWL, DCTERMS
from pyecore.ecore import EClassifier, EClass, EAttribute, EReference, EString, EObject, EEnum
from pyecore.resources import ResourceSet, URI
from bs4 import BeautifulSoup
import os

shapes_file = 'SysML-shapes.ttl'
if genOASIS: shapes_file = 'sysml-shapes.ttl'

def multiplicity(eAttribute):
    lower = eAttribute.lowerBound
    upper = eAttribute.upperBound
    if lower==1 and upper==1: return oslc['Exactly-one']
    if lower==1 and upper !=0: return oslc['One-or-many']
    if lower==0 and upper==1: return oslc['Zero-or-one']
    if lower==0 and upper !=0: return oslc['Zero-or-many']

def valueType(a):
    match a.eType.name:
        case 'Boolean': return XSD.boolean
        case 'String': return XSD.string
        case 'Real': return XSD.float
        case 'Integer': return XSD.integer
        case _: return oslc_sysml.term(a.eType.name)

def propertyAlreadyAdded(propname, eclass, shape, graph):
    # is the property already ready in this shape
    # Note: the property could be a global property definition or in a blank node
    props = list(graph.objects(shape, oslc.property))
    props = list(map(lambda s: graph.value(s, oslc.name).value, props))
    return propname in props


def addClassProperties(eclass, shape, graph, vocab_namespace, shape_namespace):
    # generate the properties specific to this ResourceShape
    for a in eclass.eStructuralFeatures:
        name = a.name
        cls = vocab_namespace.term(eclass.name)
        prop = vocab_namespace.term(name)
        # does the uninherited property appear in more than one class?
        # if it does, then add the Property constraint using a blank node for the class
        # if not, then define the property so it can be reused where it is inherited
        propval = shape_namespace.term(name)
        if propertyAlreadyAdded(name, eclass, shape, graph): return
        if name in multiDefProps:
            propval = BNode()
        graph.add((shape, oslc.property, propval))

        # and create the oslc:Property elements defined by this EClass
        graph.add((propval, RDF.type, oslc.Property))
        graph.add((propval, oslc.name, Literal(name)))
        graph.add((propval, oslc.occurs, multiplicity(a)))
        graph.add((propval, oslc.propertyDefinition, URIRef(prop)))
        graph.add((propval, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
        if isinstance(a, EReference):
            graph.add((propval, oslc.valueType, oslc.Resource))
            graph.add((propval, oslc.range, vocab_namespace.term(a.eType.name)))
            graph.add((propval, oslc.representation, oslc.Either))
        graph.add((propval, DCTERMS.description, Literal(comment(a,strip=False), datatype=RDF.XMLLiteral)))
        # is the property a redefinition? If so, remove the inherited property
        # Commented out because we decide that 1) redefined properties are inherited in SysML v2
        # and this is the behavior of the Services REST API JSON representation
        """
        redefinitions = a.getEAnnotation('redefines')
        if redefinitions is not None:
            reference = redefinitions.references[0].name
            prop = shape_namespace.term(reference)
            graph.remove((shape, oslc.property, prop))
            # the property could have been added with a blank node
        """


def addSuperclassProperties(eclass, shape, graph, vocab_namespace, shape_namespace):
    if eclass.eSuperTypes is not None and len(eclass.eSuperTypes) >= 1:
        for super in eclass.eSuperTypes:
            # recursively add the properties for the superclasses up the hierarchy
            addSuperclassProperties(super, shape, graph, vocab_namespace, shape_namespace)
            # and add the properties for this superclass
            addClassProperties(super, shape, graph, vocab_namespace, shape_namespace)

    # some useful RDF namespaces
vann = Namespace('http://purl.org/vocab/vann/')
oslc = Namespace('http://open-services.net/ns/core#')
oslc_am = Namespace('http://open-services.net/ns/am#')
oslc_sysml_shapes = Namespace('https://www.omg.org/spec/SysML/20250201/shapes/')  # OMG namespace versioned for constraints
if genOASIS: oslc_sysml_shapes = Namespace('http://open-services.net/ns/sysml/shapes/20250201#')  # OASIS namespace

jazz_am = Namespace('http://jazz.net/ns/dm/linktypes#')

# create the SysML constraints graph, initially empty
g = Graph()
g.bind('oslc_sysml', oslc_sysml)
g.bind('oslc_am', oslc_am)
g.bind('oslc', oslc)
g.bind('oslc_sysml_shapes', oslc_sysml_shapes)
g.bind('jazz_am', jazz_am)

s = URIRef(oslc_sysml_shapes)
g.add((s, RDF.type, oslc.ResourceShapeConstraints))
g.add((s, RDFS.label, Literal("OSLC  System Modeling Language (SysML) Constraints")))
g.add((s, DCTERMS.description, Literal("<p>Constraints on vocabulary terms defined in the OSLC System Modeling Language (SysML) namespace.</p>", datatype=RDF.XMLLiteral)))
g.add((s, DCTERMS.title, Literal("OSLC System Modeling Language (SysML) Version 2.0 Constraints")))

if genOASIS:
    g.add((s, DCTERMS.dateCopyrighted, Literal("2012-2024")))
    g.add((s, DCTERMS.issued, Literal("2025-02-01", datatype=XSD.date)))
    g.add((s, DCTERMS.hasVersion, Literal("PSD01")))
    g.add((s, DCTERMS.isPartOf, URIRef("https://docs.oasis-open-projects.org/oslc-op/sysml/v2.0/psd01/sysml-spec.html")))
    g.add((s, DCTERMS.license, URIRef("http://www.apache.org/licenses/LICENSE-2.0")))
    g.add((s, DCTERMS.source, URIRef("https://docs.oasis-open-projects.org/oslc-op/sysml/v2.0/psd01/sysml-shapes.ttl")))
else:
    g.add((s, DCTERMS.publisher, URIRef("https://www.omg.org")))
    g.add((s, DCTERMS.issued, Literal("2025-02-01", datatype=XSD.date)))
    g.add((s, DCTERMS.hasVersion, Literal("V2")))
    g.add((s, DCTERMS.isPartOf, URIRef('https://www.omg.org/spec/SysML')))
    g.add((s, DCTERMS.source, URIRef('https://github.com/Systems-Modeling/SysML-v2-API-Services/tree/master/public/oslc/SysML-vocab.ttl')))
    g.add((s, DCTERMS.license, URIRef('')))

# load the SysML.ecore model - this is the merged KerML and SysML metamodel using a single namespace
rset = ResourceSet()
resource = rset.get_resource(URI('/Users/jamsden/Developer/SysML-v2-Pilot-Implementation/org.omg.sysml/model/sysml_clean.ecore'))
mm_root = resource.contents[0]
rset.metamodel_registry[mm_root.nsURI] = mm_root
# At this point, the .ecore is loaded in the 'rset' as a metamodel


for c in mm_root.eClassifiers:
    # Generate the ResourceShape
    if isinstance(c, EClass):
        shape = oslc_sysml_shapes.term(c.name+'Shape')
        g.add((shape, RDF.type, oslc.ResourceShape))  # only Element
        g.add((shape, DCTERMS.title, Literal(c.name+'Shape', datatype=RDF.XMLLiteral)))
        g.add((shape, DCTERMS.description, Literal(comment(c,strip=False),datatype=RDF.XMLLiteral)))
        g.add((shape, oslc.describes, URIRef(oslc_sysml.term(c.name))))

        # add all the properties inherited from all superclasses up to Element
        addSuperclassProperties(c, shape, g, oslc_sysml, oslc_sysml_shapes)
        addClassProperties(c, shape, g, oslc_sysml, oslc_sysml_shapes)

        # add the inherited oslc_am:Resource properties
        g.add((shape, oslc.property, oslc_sysml_shapes.type))
        g.add((shape, oslc.property, oslc_sysml_shapes.dctype))
        g.add((shape, oslc.property, oslc_sysml_shapes.identifier))
        g.add((shape, oslc.property, oslc_sysml_shapes.title))
        g.add((shape, oslc.property, oslc_sysml_shapes.shortTitle))
        g.add((shape, oslc.property, oslc_sysml_shapes.description))
        g.add((shape, oslc.property, oslc_sysml_shapes.source))
        g.add((shape, oslc.property, oslc_sysml_shapes.creator))
        g.add((shape, oslc.property, oslc_sysml_shapes.created))
        g.add((shape, oslc.property, oslc_sysml_shapes.contributor))
        g.add((shape, oslc.property, oslc_sysml_shapes.modified))
        g.add((shape, oslc.property, oslc_sysml_shapes.serviceProvider))
        g.add((shape, oslc.property, oslc_sysml_shapes.instanceShape))
        g.add((shape, oslc.property, oslc_sysml_shapes.derives))
        g.add((shape, oslc.property, oslc_sysml_shapes.elaborates))
        g.add((shape, oslc.property, oslc_sysml_shapes.refine))
        g.add((shape, oslc.property, oslc_sysml_shapes.external))
        g.add((shape, oslc.property, oslc_sysml_shapes.satisfy))
        g.add((shape, oslc.property, oslc_sysml_shapes.trace))
        
    # and create the oslc_am:Resource inherited oslc:Property items
    g.add((oslc_sysml_shapes.type, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.type, oslc.name, Literal('type')))
    g.add((oslc_sysml_shapes.type, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_sysml_shapes.type, oslc.propertyDefinition, RDF.type))
    g.add((oslc_sysml_shapes.type, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.type, oslc.valueType, oslc.Resource))
    g.add((oslc_sysml_shapes.type, oslc.range, RDFS.Class))
    g.add((oslc_sysml_shapes.type, oslc.representation, oslc.Reference))
    g.add((oslc_sysml_shapes.type, DCTERMS.description, Literal("The resource type URIs.", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.dctype, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.dctype, oslc.name, Literal('dctype')))
    g.add((oslc_sysml_shapes.dctype, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_sysml_shapes.dctype, oslc.propertyDefinition, DCTERMS.type))
    g.add((oslc_sysml_shapes.dctype, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.dctype, oslc.valueType, XSD.string))
    g.add((oslc_sysml_shapes.dctype, DCTERMS.description, Literal("A short string representation for the type, for example ‘Car’.", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.identifier, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.identifier, oslc.name, Literal('identifier')))
    g.add((oslc_sysml_shapes.identifier, oslc.occurs, oslc['Exactly-one']))
    g.add((oslc_sysml_shapes.identifier, oslc.propertyDefinition, DCTERMS.identifier))
    g.add((oslc_sysml_shapes.identifier, oslc.readOnly, Literal('true',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.identifier, oslc.valueType, XSD.string))
    g.add((oslc_sysml_shapes.identifier, DCTERMS.description, Literal("""A unique identifier for a resource. Typically read-only and assigned by the
service provider when a resource is created. Not typically intended for end-user display.""", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.title, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.title, oslc.name, Literal('title')))
    g.add((oslc_sysml_shapes.title, oslc.occurs, oslc['Exactly-one']))
    g.add((oslc_sysml_shapes.title, oslc.propertyDefinition, DCTERMS.title))
    g.add((oslc_sysml_shapes.title, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.title, oslc.valueType, RDF.XMLLiteral))
    g.add((oslc_sysml_shapes.title, DCTERMS.description, Literal("Title of the resource represented as rich text in XHTML content.", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.shortTitle, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.shortTitle, oslc.name, Literal('shortTitle')))
    g.add((oslc_sysml_shapes.shortTitle, oslc.occurs, oslc['Zero-or-one']))
    g.add((oslc_sysml_shapes.shortTitle, oslc.propertyDefinition, oslc.shortTitle))
    g.add((oslc_sysml_shapes.shortTitle, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.shortTitle, oslc.valueType, RDF.XMLLiteral))
    g.add((oslc_sysml_shapes.shortTitle, DCTERMS.description, Literal("{{Short name identifying a resource, often used as an abbreviated identifier for presentation to end-users. SHOULD include only content that is valid inside an XHTML &lt;span&gt; element}}.", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.description, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.description, oslc.name, Literal('description')))
    g.add((oslc_sysml_shapes.description, oslc.occurs, oslc['Zero-or-one']))
    g.add((oslc_sysml_shapes.description, oslc.propertyDefinition, DCTERMS.description))
    g.add((oslc_sysml_shapes.description, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.description, oslc.valueType, RDF.XMLLiteral))
    g.add((oslc_sysml_shapes.description, DCTERMS.description, Literal("Descriptive text about resource represented as rich text in XHTML content.", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.source, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.source, oslc.name, Literal('source')))
    g.add((oslc_sysml_shapes.source, oslc.occurs, oslc['Zero-or-one']))
    g.add((oslc_sysml_shapes.source, oslc.propertyDefinition, DCTERMS.source))
    g.add((oslc_sysml_shapes.source, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.source, oslc.valueType, oslc.Resource))
    g.add((oslc_sysml_shapes.source, oslc.range, oslc.Any))
    g.add((oslc_sysml_shapes.source, oslc.representation, oslc.Reference))
    g.add((oslc_sysml_shapes.source, DCTERMS.description, Literal("The resource URI a client can perform a get on to obtain the original non-OSLC AM formatted resource that was used to create this resource. The source resource is usually a binary or proprietary format that the service provider can consume and convert into an OSLC AM format. The service may use content negotiation with the Accept header to obtain the desired content type.", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.creator, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.creator, oslc.name, Literal('creator')))
    g.add((oslc_sysml_shapes.creator, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_sysml_shapes.creator, oslc.propertyDefinition, DCTERMS.creator))
    g.add((oslc_sysml_shapes.creator, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.creator, oslc.valueType, oslc.AnyResource))
    g.add((oslc_sysml_shapes.creator, oslc.range, oslc.Any))
    g.add((oslc_sysml_shapes.creator, oslc.representation, oslc.Either))
    g.add((oslc_sysml_shapes.creator, DCTERMS.description, Literal("Creator or creators of the resource. It is likely that the target resource will be a foaf:Person but that is not necessarily the case.", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.created, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.created, oslc.name, Literal('created')))
    g.add((oslc_sysml_shapes.created, oslc.occurs, oslc['Zero-or-one']))
    g.add((oslc_sysml_shapes.created, oslc.propertyDefinition, DCTERMS.created))
    g.add((oslc_sysml_shapes.created, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.created, oslc.valueType, XSD.dateTime))
    g.add((oslc_sysml_shapes.created, DCTERMS.description, Literal("Timestamp of resource creation.", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.contributor, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.contributor, oslc.name, Literal('contributor')))
    g.add((oslc_sysml_shapes.contributor, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_sysml_shapes.contributor, oslc.propertyDefinition, DCTERMS.contributor))
    g.add((oslc_sysml_shapes.contributor, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.contributor, oslc.valueType, oslc.AnyResource))
    g.add((oslc_sysml_shapes.contributor, oslc.range, oslc.Any))
    g.add((oslc_sysml_shapes.contributor, oslc.representation, oslc.Either))
    g.add((oslc_sysml_shapes.contributor, DCTERMS.description, Literal("Contributor or contributors to the resource. It is likely that the target resource will be a foaf:Person but that is not necessarily the case.", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.modified, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.modified, oslc.name, Literal('modified')))
    g.add((oslc_sysml_shapes.modified, oslc.occurs, oslc['Zero-or-one']))
    g.add((oslc_sysml_shapes.modified, oslc.propertyDefinition, DCTERMS.modified))
    g.add((oslc_sysml_shapes.modified, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.modified, oslc.valueType, XSD.dateTime))
    g.add((oslc_sysml_shapes.modified, DCTERMS.description, Literal("Timestamp of latest resource modification.", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.serviceProvider, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.serviceProvider, oslc.name, Literal('serviceProvider')))
    g.add((oslc_sysml_shapes.serviceProvider, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_sysml_shapes.serviceProvider, oslc.propertyDefinition, oslc.serviceProvider))
    g.add((oslc_sysml_shapes.serviceProvider, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.serviceProvider, oslc.valueType, oslc.Resource))
    g.add((oslc_sysml_shapes.serviceProvider, oslc.range, oslc.ServiceProvider))
    g.add((oslc_sysml_shapes.serviceProvider, oslc.representation, oslc.Reference))
    g.add((oslc_sysml_shapes.serviceProvider, DCTERMS.description, Literal("""A link to the resource's OSLC Service Provider. There may be cases when the
subject resource is available from a service provider that implements multiple domain
specifications, which could result in multiple values for this property.""", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.instanceShape, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.instanceShape, oslc.name, Literal('instanceShape')))
    g.add((oslc_sysml_shapes.instanceShape, oslc.occurs, oslc['Zero-or-one']))
    g.add((oslc_sysml_shapes.instanceShape, oslc.propertyDefinition, oslc.instanceShape))
    g.add((oslc_sysml_shapes.instanceShape, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.instanceShape, oslc.valueType, oslc.Resource))
    g.add((oslc_sysml_shapes.instanceShape, oslc.range, oslc.ResourceShape))
    g.add((oslc_sysml_shapes.instanceShape, oslc.representation, oslc.Reference))
    g.add((oslc_sysml_shapes.instanceShape, DCTERMS.description, Literal("""The URI of a Resource Shape that describes the possible properties, occurrence,
value types, allowed values and labels. This shape information is useful in displaying the subject
resource as well as guiding clients in performing modifications. Instance shapes may be specific
to the authenticated user associated with the request that retrieved the resource, the current
state of the resource and other factors and thus should not be cached.""", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.derives, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.derives, oslc.name, Literal('derives')))
    g.add((oslc_sysml_shapes.derives, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_sysml_shapes.derives, oslc.propertyDefinition, jazz_am.derives))
    g.add((oslc_sysml_shapes.derives, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.derives, oslc.valueType, oslc.Resource))
    g.add((oslc_sysml_shapes.derives, oslc.range, oslc.Any))
    g.add((oslc_sysml_shapes.derives, oslc.representation, oslc.Reference))
    g.add((oslc_sysml_shapes.derives, DCTERMS.description, Literal("""The resource that derives from another resource originated from or is
significantly influenced by the referenced resource. For example a model element derives from a
requirement.""", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.elaborates, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.elaborates, oslc.name, Literal('elaborates')))
    g.add((oslc_sysml_shapes.elaborates, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_sysml_shapes.elaborates, oslc.propertyDefinition, jazz_am.elaborates))
    g.add((oslc_sysml_shapes.elaborates, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.elaborates, oslc.valueType, oslc.Resource))
    g.add((oslc_sysml_shapes.elaborates, oslc.range, oslc.Any))
    g.add((oslc_sysml_shapes.elaborates, oslc.representation, oslc.Reference))
    g.add((oslc_sysml_shapes.elaborates, DCTERMS.description, Literal("""This resource elaborates the referenced resource.""", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.refine, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.refine, oslc.name, Literal('refine')))
    g.add((oslc_sysml_shapes.refine, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_sysml_shapes.refine, oslc.propertyDefinition, jazz_am.refine))
    g.add((oslc_sysml_shapes.refine, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.refine, oslc.valueType, oslc.Resource))
    g.add((oslc_sysml_shapes.refine, oslc.range, oslc.Any))
    g.add((oslc_sysml_shapes.refine, oslc.representation, oslc.Reference))
    g.add((oslc_sysml_shapes.refine, DCTERMS.description, Literal("""The target is a refinement of the source. (e.g. a use case scenario
might be a refinement of a textual requirement that describes the interaction).""", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.external, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.external, oslc.name, Literal('external')))
    g.add((oslc_sysml_shapes.external, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_sysml_shapes.external, oslc.propertyDefinition, jazz_am.external))
    g.add((oslc_sysml_shapes.external, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.external, oslc.valueType, oslc.Resource))
    g.add((oslc_sysml_shapes.external, oslc.range, oslc.Any))
    g.add((oslc_sysml_shapes.external, oslc.representation, oslc.Reference))
    g.add((oslc_sysml_shapes.external, DCTERMS.description, Literal("""A generic link from a resource to an external web page.""", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.satisfy, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.satisfy, oslc.name, Literal('satisfy')))
    g.add((oslc_sysml_shapes.satisfy, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_sysml_shapes.satisfy, oslc.propertyDefinition, jazz_am.satisfy))
    g.add((oslc_sysml_shapes.satisfy, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.satisfy, oslc.valueType, oslc.Resource))
    g.add((oslc_sysml_shapes.satisfy, oslc.range, oslc.Any))
    g.add((oslc_sysml_shapes.satisfy, oslc.representation, oslc.Reference))
    g.add((oslc_sysml_shapes.satisfy, DCTERMS.description, Literal("""The model element satisfies the requirement (e.g. The use case
satisfies a functional requirement).""", datatype=RDF.XMLLiteral)))

    g.add((oslc_sysml_shapes.trace, RDF.type, oslc.Property))
    g.add((oslc_sysml_shapes.trace, oslc.name, Literal('trace')))
    g.add((oslc_sysml_shapes.trace, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_sysml_shapes.trace, oslc.propertyDefinition, jazz_am.trace))
    g.add((oslc_sysml_shapes.trace, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_sysml_shapes.trace, oslc.valueType, oslc.Resource))
    g.add((oslc_sysml_shapes.trace, oslc.range, oslc.Any))
    g.add((oslc_sysml_shapes.trace, oslc.representation, oslc.Reference))
    g.add((oslc_sysml_shapes.trace, DCTERMS.description, Literal("""The model element has a trace to the requirement (e.g. An attribute
or its value are traced to a requirement).""", datatype=RDF.XMLLiteral)))


g.serialize(destination=f'../{shapes_file}')

if genOASIS:  # add the copywrite header
    with open('../sysml-shapes.ttl','r') as f:
        with open('../temp.ttl','w') as f2: 
            f2.write(copyright)
            f2.write(f.read())
    os.remove('../sysml-shapes.ttl')
    os.rename('../temp.ttl','../sysml-shapes.ttl')

## Get SysML v2 classes
A notebook to get a list of the SysML v2 classes and generate the vocabToShape div sections needed in sysml-shapes.html.



In [5]:
# Read and parse the sysml-vocab.ttl file
from rdflib import Graph, URIRef, Literal, Namespace, RDF

# some useful RDF namespaces
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace('http://www.w3.org/2000/01/rdf-schema#')
DCTERMS = Namespace('http://purl.org/dc/terms/')
oslc = Namespace('http://open-services.net/ns/core#')
oslc_am = Namespace('http://open-services.net/ns/am#')

g = Graph()
g.parse('../SysML-vocab.ttl')


<Graph identifier=N99bfa89dc90247038d7de2d2494cccc6 (<class 'rdflib.graph.Graph'>)>

In [6]:
# get a list of the classes and create the dev elements needed for shapeToSpec
classes = g.subjects(rdf.type, rdfs.Class)

# and now create the shapeToSpec div for each class
for c in classes:
    name = str(c)
    name = name[name.index('#')+1:]
    print(
f"""

<div
        title="Constraints for {name}"
        data-include="./SysML-shapes.ttl#{name}Shape"
        data-oninclude="shapeToSpec"
        data-include-sync="true"
        data-include-replace="true"
        data-include-format="html"
      ></div>
"""        
    )



<div
        title="Constraints for AcceptActionUsage"
        data-include="./SysML-shapes.ttl#AcceptActionUsageShape"
        data-oninclude="shapeToSpec"
        data-include-sync="true"
        data-include-replace="true"
        data-include-format="html"
      ></div>



<div
        title="Constraints for ActionDefinition"
        data-include="./SysML-shapes.ttl#ActionDefinitionShape"
        data-oninclude="shapeToSpec"
        data-include-sync="true"
        data-include-replace="true"
        data-include-format="html"
      ></div>



<div
        title="Constraints for ActionUsage"
        data-include="./SysML-shapes.ttl#ActionUsageShape"
        data-oninclude="shapeToSpec"
        data-include-sync="true"
        data-include-replace="true"
        data-include-format="html"
      ></div>



<div
        title="Constraints for ActorMembership"
        data-include="./SysML-shapes.ttl#ActorMembershipShape"
        data-oninclude="shapeToSpec"
        data-include-sync

# Generate OSLC KerML vocab

In [ ]:
# create the KerML vocabulary graph, initially empty

oslc_kerml = Namespace('https://www.omg.org/spec/kerml/vocabulary#')

g = Graph()
g.bind('oslc_kerml', oslc_kerml)
g.bind('oslc_am', oslc_am)

# add the vocabulary ontology
o = URIRef(oslc_kerml)
g.add((o, RDF.type, OWL.Ontology))
g.add((o, RDFS.label, Literal("OSLC KerML Vocabulary")))
g.add((o, DCTERMS.description, Literal("All vocabulary URIs defined in the OSLC KerML namespace.", datatype=RDF.XMLLiteral)))
g.add((o, DCTERMS.issued, Literal("2025-02-01", datatype=XSD.date)))
g.add((o, DCTERMS.license, URIRef('')))
g.add((o, DCTERMS.publisher,  URIRef('https://www.omg.org')))
g.add((o, DCTERMS.title, Literal("OSLC KerML Vocabulary")))
g.add((o, vann.preferredNamespacePrefix, Literal("oslc_kerml")))
g.add((o, DCTERMS.hasVersion, Literal("V2")))
g.add((o, DCTERMS.isPartOf, URIRef('https://www.omg.org/spec/KerML')))
g.add((o, DCTERMS.source, URIRef('https://github.com/Systems-Modeling/SysML-v2-API-Services/tree/master/public/oslc/KerML-vocab.ttl')))

# load the KerML.ecore model
rset = ResourceSet()
resource = rset.get_resource(URI('/Users/jamsden/Developer/SysML-v2-Pilot-Implementation/org.omg.sysml/model/KerML.ecore'))
mm_root = resource.contents[0]
rset.metamodel_registry[mm_root.nsURI] = mm_root
# At this point, the .ecore is loaded in the 'rset' as a metamodel


for c in mm_root.eClassifiers:
    # Generate the classes
    if isinstance(c, EClass):
        g.add((oslc_kerml.term(c.name), RDF.type, RDFS.Class))
        if c.name=='Element':
            g.add((oslc_kerml.term(c.name), RDFS.subClassOf, oslc_am.Resource))  # only Element
        if c.eSuperTypes is not None and len(c.eSuperTypes) >= 1:
            for super in c.eSuperTypes:
                g.add((oslc_kerml.term(c.name), RDFS.subClassOf, oslc_kerml.term(super.name)))
        g.add((oslc_kerml.term(c.name), RDFS.label, Literal(c.name)))
        g.add((oslc_kerml.term(c.name), RDFS.comment, Literal(comment(c))))
        g.add((oslc_kerml.term(c.name), RDFS.isDefinedBy, URIRef(oslc_kerml)))

        # generate the properties
        for a in c.eStructuralFeatures:
            name = a.name  # use the class name as a prefix to the attribute name to ensure they are unique
            s = oslc_kerml.term(name)
            g.add((s, RDF.type, RDF.Property))
            g.add((s, RDFS.label, Literal(name)))
            pcomment = g.value(s, RDFS.comment)
            if pcomment is not None:
                # This is a property with multiple domains
                # Concatenate the comment for each domain class into a single comment
                pcomment = pcomment + '\n' + c.name+': '+comment(a)
                g.remove((s, RDFS.comment, None))
                g.add((s, RDFS.comment, Literal(pcomment)))
            else:
                g.add((s, RDFS.comment, Literal(c.name+': '+comment(a))))
            g.add((s, RDFS.isDefinedBy, URIRef(oslc_kerml)))
    if isinstance(c, EEnum):
        g.add((oslc_kerml.term(c.name), RDF.type, RDFS.Class))
        g.add((oslc_kerml.term(c.name), RDFS.label, Literal(c.name)))
        g.add((oslc_kerml.term(c.name), RDFS.comment, Literal(comment(c))))
        g.add((oslc_kerml.term(c.name), RDFS.isDefinedBy, URIRef(oslc_kerml)))

        # generate the enumeration literals
        for a in c.eLiterals:
            name = a.name  # don't use the class name as a prefix to the attribute name to ensure they are unique
            s = oslc_kerml.term(name)
            g.add((s, RDF.type, oslc_kerml.term(c.name)))
            g.add((s, RDFS.label, Literal(name)))
            g.add((s, RDFS.comment, Literal(comment(a))))
            g.add((s, RDFS.isDefinedBy, URIRef(oslc_kerml)))
        
g.serialize(destination='../KerML-vocab.ttl')



<Graph identifier=Nd9d2c7f624334c759cce0aeb8451c365 (<class 'rdflib.graph.Graph'>)>

In [ ]:
# Generate KerML Constraints

    

# create the SysML constraints graph, initially empty
oslc_kerml_shapes = Namespace('https://www.omg.org/spec/kerml/20250201/shapes#')
jazz_am = Namespace('http://jazz.net/ns/dm/linktypes#')

g = Graph()
g.bind('oslc_kerml', oslc_kerml)
g.bind('oslc_am', oslc_am)
g.bind('oslc', oslc)
g.bind('', oslc_kerml_shapes)
g.bind('jazz_am', jazz_am)

s = URIRef(oslc_kerml_shapes)
g.add((s, RDF.type, oslc.ResourceShapeConstraints))
g.add((s, RDFS.label, Literal("OSLC  Kernel Modeling Language (KerML) Constraints")))
g.add((s, DCTERMS.dateCopyrighted, Literal("2012-2024")))
g.add((s, DCTERMS.description, Literal("<p>Constraints on vocabulary terms defined in the OSLC Kernel Modeling Language (KerML) namespace.</p>", datatype=RDF.XMLLiteral)))
g.add((s, DCTERMS.publisher,  URIRef('https://www.omg.org')))
g.add((s, DCTERMS.title, Literal("OSLC Kernel Modeling Language (SysML) Version 2.0 Constraints")))
g.add((s, DCTERMS.hasVersion, Literal("V2")))
g.add((s, DCTERMS.isPartOf, URIRef('https://www.omg.org/spec/KerML')))
g.add((s, DCTERMS.source, URIRef('https://github.com/Systems-Modeling/SysML-v2-API-Services/tree/master/public/oslc/KerML-vocab.ttl')))
g.add((s, DCTERMS.license, URIRef('')))

for c in mm_root.eClassifiers:
    # Generate the ResourceShape
    if isinstance(c, EClass):
        shape = oslc_kerml_shapes.term(c.name+'Shape')
        g.add((shape, RDF.type, oslc.ResourceShape))  # only Element
        g.add((shape, DCTERMS.title, Literal(c.name+'Shape', datatype=RDF.XMLLiteral)))
        g.add((shape, DCTERMS.description, Literal(comment(c,strip=False),datatype=RDF.XMLLiteral)))
        g.add((shape, oslc.describes, URIRef(oslc_kerml.term(c.name))))

        # add all the properties inherited from all superclasses up to Element
        addSuperclassProperties(c, shape, g, oslc_kerml, oslc_kerml_shapes)
        addClassProperties(c, shape, g, oslc_kerml, oslc_kerml_shapes)

        # add the inherited oslc_am:Resource properties
        g.add((shape, oslc.property, oslc_kerml_shapes.type))
        g.add((shape, oslc.property, oslc_kerml_shapes.dctype))
        g.add((shape, oslc.property, oslc_kerml_shapes.identifier))
        g.add((shape, oslc.property, oslc_kerml_shapes.title))
        g.add((shape, oslc.property, oslc_kerml_shapes.shortTitle))
        g.add((shape, oslc.property, oslc_kerml_shapes.description))
        g.add((shape, oslc.property, oslc_kerml_shapes.source))
        g.add((shape, oslc.property, oslc_kerml_shapes.creator))
        g.add((shape, oslc.property, oslc_kerml_shapes.created))
        g.add((shape, oslc.property, oslc_kerml_shapes.contributor))
        g.add((shape, oslc.property, oslc_kerml_shapes.modified))
        g.add((shape, oslc.property, oslc_kerml_shapes.serviceProvider))
        g.add((shape, oslc.property, oslc_kerml_shapes.instanceShape))
        g.add((shape, oslc.property, oslc_kerml_shapes.derives))
        g.add((shape, oslc.property, oslc_kerml_shapes.elaborates))
        g.add((shape, oslc.property, oslc_kerml_shapes.refine))
        g.add((shape, oslc.property, oslc_kerml_shapes.external))
        g.add((shape, oslc.property, oslc_kerml_shapes.satisfy))
        g.add((shape, oslc.property, oslc_kerml_shapes.trace))
        
    # and create the oslc_am:Resource inherited oslc:Property items
    g.add((oslc_kerml_shapes.type, RDF.type, oslc.Property))
    g.add((oslc_kerml_shapes.type, oslc.name, Literal('type')))
    g.add((oslc_kerml_shapes.type, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_kerml_shapes.type, oslc.propertyDefinition, RDF.type))
    g.add((oslc_kerml_shapes.type, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_kerml_shapes.type, oslc.valueType, oslc.Resource))
    g.add((oslc_kerml_shapes.type, oslc.range, RDFS.Class))
    g.add((oslc_kerml_shapes.type, oslc.representation, oslc.Reference))
    g.add((oslc_kerml_shapes.type, DCTERMS.description, Literal("The resource type URIs.", datatype=RDF.XMLLiteral)))

    g.add((oslc_kerml_shapes.dctype, RDF.type, oslc.Property))
    g.add((oslc_kerml_shapes.dctype, oslc.name, Literal('dctype')))
    g.add((oslc_kerml_shapes.dctype, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_kerml_shapes.dctype, oslc.propertyDefinition, DCTERMS.type))
    g.add((oslc_kerml_shapes.dctype, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_kerml_shapes.dctype, oslc.valueType, XSD.string))
    g.add((oslc_kerml_shapes.dctype, DCTERMS.description, Literal("A short string representation for the type, for example ‘Car’.", datatype=RDF.XMLLiteral)))

    g.add((oslc_kerml_shapes.identifier, RDF.type, oslc.Property))
    g.add((oslc_kerml_shapes.identifier, oslc.name, Literal('identifier')))
    g.add((oslc_kerml_shapes.identifier, oslc.occurs, oslc['Exactly-one']))
    g.add((oslc_kerml_shapes.identifier, oslc.propertyDefinition, DCTERMS.identifier))
    g.add((oslc_kerml_shapes.identifier, oslc.readOnly, Literal('true',datatype=XSD.boolean)))
    g.add((oslc_kerml_shapes.identifier, oslc.valueType, XSD.string))
    g.add((oslc_kerml_shapes.identifier, DCTERMS.description, Literal("""A unique identifier for a resource. Typically read-only and assigned by the
service provider when a resource is created. Not typically intended for end-user display.""", datatype=RDF.XMLLiteral)))

    g.add((oslc_kerml_shapes.title, RDF.type, oslc.Property))
    g.add((oslc_kerml_shapes.title, oslc.name, Literal('title')))
    g.add((oslc_kerml_shapes.title, oslc.occurs, oslc['Exactly-one']))
    g.add((oslc_kerml_shapes.title, oslc.propertyDefinition, DCTERMS.title))
    g.add((oslc_kerml_shapes.title, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_kerml_shapes.title, oslc.valueType, RDF.XMLLiteral))
    g.add((oslc_kerml_shapes.title, DCTERMS.description, Literal("Title of the resource represented as rich text in XHTML content.", datatype=RDF.XMLLiteral)))

    g.add((oslc_kerml_shapes.shortTitle, RDF.type, oslc.Property))
    g.add((oslc_kerml_shapes.shortTitle, oslc.name, Literal('shortTitle')))
    g.add((oslc_kerml_shapes.shortTitle, oslc.occurs, oslc['Zero-or-one']))
    g.add((oslc_kerml_shapes.shortTitle, oslc.propertyDefinition, oslc.shortTitle))
    g.add((oslc_kerml_shapes.shortTitle, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_kerml_shapes.shortTitle, oslc.valueType, RDF.XMLLiteral))
    g.add((oslc_kerml_shapes.shortTitle, DCTERMS.description, Literal("{{Short name identifying a resource, often used as an abbreviated identifier for presentation to end-users. SHOULD include only content that is valid inside an XHTML &lt;span&gt; element}}.", datatype=RDF.XMLLiteral)))

    g.add((oslc_kerml_shapes.description, RDF.type, oslc.Property))
    g.add((oslc_kerml_shapes.description, oslc.name, Literal('description')))
    g.add((oslc_kerml_shapes.description, oslc.occurs, oslc['Zero-or-one']))
    g.add((oslc_kerml_shapes.description, oslc.propertyDefinition, DCTERMS.description))
    g.add((oslc_kerml_shapes.description, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_kerml_shapes.description, oslc.valueType, RDF.XMLLiteral))
    g.add((oslc_kerml_shapes.description, DCTERMS.description, Literal("Descriptive text about resource represented as rich text in XHTML content.", datatype=RDF.XMLLiteral)))

    g.add((oslc_kerml_shapes.source, RDF.type, oslc.Property))
    g.add((oslc_kerml_shapes.source, oslc.name, Literal('source')))
    g.add((oslc_kerml_shapes.source, oslc.occurs, oslc['Zero-or-one']))
    g.add((oslc_kerml_shapes.source, oslc.propertyDefinition, DCTERMS.source))
    g.add((oslc_kerml_shapes.source, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_kerml_shapes.source, oslc.valueType, oslc.Resource))
    g.add((oslc_kerml_shapes.source, oslc.range, oslc.Any))
    g.add((oslc_kerml_shapes.source, oslc.representation, oslc.Reference))
    g.add((oslc_kerml_shapes.source, DCTERMS.description, Literal("The resource URI a client can perform a get on to obtain the original non-OSLC AM formatted resource that was used to create this resource. The source resource is usually a binary or proprietary format that the service provider can consume and convert into an OSLC AM format. The service may use content negotiation with the Accept header to obtain the desired content type.", datatype=RDF.XMLLiteral)))

    g.add((oslc_kerml_shapes.creator, RDF.type, oslc.Property))
    g.add((oslc_kerml_shapes.creator, oslc.name, Literal('creator')))
    g.add((oslc_kerml_shapes.creator, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_kerml_shapes.creator, oslc.propertyDefinition, DCTERMS.creator))
    g.add((oslc_kerml_shapes.creator, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_kerml_shapes.creator, oslc.valueType, oslc.AnyResource))
    g.add((oslc_kerml_shapes.creator, oslc.range, oslc.Any))
    g.add((oslc_kerml_shapes.creator, oslc.representation, oslc.Either))
    g.add((oslc_kerml_shapes.creator, DCTERMS.description, Literal("Creator or creators of the resource. It is likely that the target resource will be a foaf:Person but that is not necessarily the case.", datatype=RDF.XMLLiteral)))

    g.add((oslc_kerml_shapes.created, RDF.type, oslc.Property))
    g.add((oslc_kerml_shapes.created, oslc.name, Literal('created')))
    g.add((oslc_kerml_shapes.created, oslc.occurs, oslc['Zero-or-one']))
    g.add((oslc_kerml_shapes.created, oslc.propertyDefinition, DCTERMS.created))
    g.add((oslc_kerml_shapes.created, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_kerml_shapes.created, oslc.valueType, XSD.dateTime))
    g.add((oslc_kerml_shapes.created, DCTERMS.description, Literal("Timestamp of resource creation.", datatype=RDF.XMLLiteral)))

    g.add((oslc_kerml_shapes.contributor, RDF.type, oslc.Property))
    g.add((oslc_kerml_shapes.contributor, oslc.name, Literal('contributor')))
    g.add((oslc_kerml_shapes.contributor, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_kerml_shapes.contributor, oslc.propertyDefinition, DCTERMS.contributor))
    g.add((oslc_kerml_shapes.contributor, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_kerml_shapes.contributor, oslc.valueType, oslc.AnyResource))
    g.add((oslc_kerml_shapes.contributor, oslc.range, oslc.Any))
    g.add((oslc_kerml_shapes.contributor, oslc.representation, oslc.Either))
    g.add((oslc_kerml_shapes.contributor, DCTERMS.description, Literal("Contributor or contributors to the resource. It is likely that the target resource will be a foaf:Person but that is not necessarily the case.", datatype=RDF.XMLLiteral)))

    g.add((oslc_kerml_shapes.modified, RDF.type, oslc.Property))
    g.add((oslc_kerml_shapes.modified, oslc.name, Literal('modified')))
    g.add((oslc_kerml_shapes.modified, oslc.occurs, oslc['Zero-or-one']))
    g.add((oslc_kerml_shapes.modified, oslc.propertyDefinition, DCTERMS.modified))
    g.add((oslc_kerml_shapes.modified, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_kerml_shapes.modified, oslc.valueType, XSD.dateTime))
    g.add((oslc_kerml_shapes.modified, DCTERMS.description, Literal("Timestamp of latest resource modification.", datatype=RDF.XMLLiteral)))

    g.add((oslc_kerml_shapes.serviceProvider, RDF.type, oslc.Property))
    g.add((oslc_kerml_shapes.serviceProvider, oslc.name, Literal('serviceProvider')))
    g.add((oslc_kerml_shapes.serviceProvider, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_kerml_shapes.serviceProvider, oslc.propertyDefinition, oslc.serviceProvider))
    g.add((oslc_kerml_shapes.serviceProvider, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_kerml_shapes.serviceProvider, oslc.valueType, oslc.Resource))
    g.add((oslc_kerml_shapes.serviceProvider, oslc.range, oslc.ServiceProvider))
    g.add((oslc_kerml_shapes.serviceProvider, oslc.representation, oslc.Reference))
    g.add((oslc_kerml_shapes.serviceProvider, DCTERMS.description, Literal("""A link to the resource's OSLC Service Provider. There may be cases when the
subject resource is available from a service provider that implements multiple domain
specifications, which could result in multiple values for this property.""", datatype=RDF.XMLLiteral)))

    g.add((oslc_kerml_shapes.instanceShape, RDF.type, oslc.Property))
    g.add((oslc_kerml_shapes.instanceShape, oslc.name, Literal('instanceShape')))
    g.add((oslc_kerml_shapes.instanceShape, oslc.occurs, oslc['Zero-or-one']))
    g.add((oslc_kerml_shapes.instanceShape, oslc.propertyDefinition, oslc.instanceShape))
    g.add((oslc_kerml_shapes.instanceShape, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_kerml_shapes.instanceShape, oslc.valueType, oslc.Resource))
    g.add((oslc_kerml_shapes.instanceShape, oslc.range, oslc.ResourceShape))
    g.add((oslc_kerml_shapes.instanceShape, oslc.representation, oslc.Reference))
    g.add((oslc_kerml_shapes.instanceShape, DCTERMS.description, Literal("""The URI of a Resource Shape that describes the possible properties, occurrence,
value types, allowed values and labels. This shape information is useful in displaying the subject
resource as well as guiding clients in performing modifications. Instance shapes may be specific
to the authenticated user associated with the request that retrieved the resource, the current
state of the resource and other factors and thus should not be cached.""", datatype=RDF.XMLLiteral)))

    g.add((oslc_kerml_shapes.derives, RDF.type, oslc.Property))
    g.add((oslc_kerml_shapes.derives, oslc.name, Literal('derives')))
    g.add((oslc_kerml_shapes.derives, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_kerml_shapes.derives, oslc.propertyDefinition, jazz_am.derives))
    g.add((oslc_kerml_shapes.derives, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_kerml_shapes.derives, oslc.valueType, oslc.Resource))
    g.add((oslc_kerml_shapes.derives, oslc.range, oslc.Any))
    g.add((oslc_kerml_shapes.derives, oslc.representation, oslc.Reference))
    g.add((oslc_kerml_shapes.derives, DCTERMS.description, Literal("""The resource that derives from another resource originated from or is
significantly influenced by the referenced resource. For example a model element derives from a
requirement.""", datatype=RDF.XMLLiteral)))

    g.add((oslc_kerml_shapes.elaborates, RDF.type, oslc.Property))
    g.add((oslc_kerml_shapes.elaborates, oslc.name, Literal('elaborates')))
    g.add((oslc_kerml_shapes.elaborates, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_kerml_shapes.elaborates, oslc.propertyDefinition, jazz_am.elaborates))
    g.add((oslc_kerml_shapes.elaborates, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_kerml_shapes.elaborates, oslc.valueType, oslc.Resource))
    g.add((oslc_kerml_shapes.elaborates, oslc.range, oslc.Any))
    g.add((oslc_kerml_shapes.elaborates, oslc.representation, oslc.Reference))
    g.add((oslc_kerml_shapes.elaborates, DCTERMS.description, Literal("""This resource elaborates the referenced resource.""", datatype=RDF.XMLLiteral)))

    g.add((oslc_kerml_shapes.refine, RDF.type, oslc.Property))
    g.add((oslc_kerml_shapes.refine, oslc.name, Literal('refine')))
    g.add((oslc_kerml_shapes.refine, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_kerml_shapes.refine, oslc.propertyDefinition, jazz_am.refine))
    g.add((oslc_kerml_shapes.refine, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_kerml_shapes.refine, oslc.valueType, oslc.Resource))
    g.add((oslc_kerml_shapes.refine, oslc.range, oslc.Any))
    g.add((oslc_kerml_shapes.refine, oslc.representation, oslc.Reference))
    g.add((oslc_kerml_shapes.refine, DCTERMS.description, Literal("""The target is a refinement of the source. (e.g. a use case scenario
might be a refinement of a textual requirement that describes the interaction).""", datatype=RDF.XMLLiteral)))

    g.add((oslc_kerml_shapes.external, RDF.type, oslc.Property))
    g.add((oslc_kerml_shapes.external, oslc.name, Literal('external')))
    g.add((oslc_kerml_shapes.external, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_kerml_shapes.external, oslc.propertyDefinition, jazz_am.external))
    g.add((oslc_kerml_shapes.external, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_kerml_shapes.external, oslc.valueType, oslc.Resource))
    g.add((oslc_kerml_shapes.external, oslc.range, oslc.Any))
    g.add((oslc_kerml_shapes.external, oslc.representation, oslc.Reference))
    g.add((oslc_kerml_shapes.external, DCTERMS.description, Literal("""A generic link from a resource to an external web page.""", datatype=RDF.XMLLiteral)))

    g.add((oslc_kerml_shapes.satisfy, RDF.type, oslc.Property))
    g.add((oslc_kerml_shapes.satisfy, oslc.name, Literal('satisfy')))
    g.add((oslc_kerml_shapes.satisfy, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_kerml_shapes.satisfy, oslc.propertyDefinition, jazz_am.satisfy))
    g.add((oslc_kerml_shapes.satisfy, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_kerml_shapes.satisfy, oslc.valueType, oslc.Resource))
    g.add((oslc_kerml_shapes.satisfy, oslc.range, oslc.Any))
    g.add((oslc_kerml_shapes.satisfy, oslc.representation, oslc.Reference))
    g.add((oslc_kerml_shapes.satisfy, DCTERMS.description, Literal("""The model element satisfies the requirement (e.g. The use case
satisfies a functional requirement).""", datatype=RDF.XMLLiteral)))

    g.add((oslc_kerml_shapes.trace, RDF.type, oslc.Property))
    g.add((oslc_kerml_shapes.trace, oslc.name, Literal('trace')))
    g.add((oslc_kerml_shapes.trace, oslc.occurs, oslc['Zero-or-many']))
    g.add((oslc_kerml_shapes.trace, oslc.propertyDefinition, jazz_am.trace))
    g.add((oslc_kerml_shapes.trace, oslc.readOnly, Literal('false',datatype=XSD.boolean)))
    g.add((oslc_kerml_shapes.trace, oslc.valueType, oslc.Resource))
    g.add((oslc_kerml_shapes.trace, oslc.range, oslc.Any))
    g.add((oslc_kerml_shapes.trace, oslc.representation, oslc.Reference))
    g.add((oslc_kerml_shapes.trace, DCTERMS.description, Literal("""The model element has a trace to the requirement (e.g. An attribute
or its value are traced to a requirement).""", datatype=RDF.XMLLiteral)))


g.serialize(destination='../KerML-shapes.ttl')



<Graph identifier=N1c9b988e5a4648dfba27a16c770d9f7e (<class 'rdflib.graph.Graph'>)>